<a href="https://colab.research.google.com/github/ashwinvaswani/Fake-News-Detection/blob/master/code/Fake-News_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# NUS Fake news detection

In [0]:
# This is the solution to the fake news classification problem

In [0]:
# Importing Libraries

In [0]:
import numpy as np
import pandas as pd
from gensim.models import Word2Vec
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier

import pickle
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import string
import random
import seaborn as sn
from sklearn.model_selection import *
from sklearn.preprocessing import *
from sklearn.linear_model import *
from sklearn.ensemble import *
from sklearn.metrics import *
from sklearn.externals import *
from sklearn.naive_bayes import *
from sklearn.svm import *
from sklearn.utils import shuffle
from textblob.classifiers import NaiveBayesClassifier
from dateutil.parser import parse
import datetime as dt
pd.options.mode.chained_assignment = None  # default='warn'
from nltk.corpus import stopwords
from nltk import *
from textblob import TextBlob, Word
import re
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [0]:
# Multi class classification :

In [0]:
def cleaner(df, b=True):
    df.columns = ['label', 'statement', 'subject', 'speaker', 'speakerTitle', 'state', 'party',
                  'barely-true', 'false', 'half-true', 'mostly-true', 'pants-fire', 'context', 'justification']
    
    stemmer = nltk.stem.PorterStemmer()
    Stopwords = set(nltk.corpus.stopwords.words('english'))

    values_nan = {'subject': 'unknown', 'speaker': 'unknown', 'speakerTitle': 'unknown', 'state': 'unknown',
                  'party': 'unknown', 'barely-true': 0.0, 'false': 0.0, 'half-true': 0.0, 'mostly-true': 0.0,
                  'pants-fire': 0.0, 'context': 'unknown', 'justification': 'unknown'}
    df.fillna(value=values_nan, inplace=True)

    if b:
        df.label.replace(
            {'half-true': 1, 'mostly-true': 1, 'false': 0, 'true': 1, 'barely-true': 0, 'pants-fire': 0},
            inplace=True)
    else:
        df.label.replace(
            {'half-true': 2, 'mostly-true': 1, 'false': 4, 'true': 0, 'barely-true': 3, 'pants-fire': 5},
            inplace=True)

    df['statement'] = df['statement'].apply(lambda x: ' '.join([word for word in x.split() if word not in Stopwords]))
    df['statement'] = df['statement'].apply(lambda x: stemmer.stem(x))

    df['context'] = df['context'].apply(lambda x: ' '.join([word for word in x.split() if word not in Stopwords]))
    df['context'] = df['context'].apply(lambda x: stemmer.stem(x))

    df['justification'] = df['justification'].apply(lambda x: ' '.join([word for word in x.split() if word not in Stopwords]))
    df['justification'] = df['justification'].apply(lambda x: stemmer.stem(x))

    df['speaker'] = df['speaker'].apply(lambda x: x.replace('-', ''))
    speaker_dict = dict()
    for i, speaker in enumerate(pd.unique(df['speaker'])):
        speaker_dict[speaker] = i

    df.speaker.replace(speaker_dict, inplace=True)

    speaker_title_dict = dict()
    for i, speakerTitle in enumerate(pd.unique(df['speakerTitle'])):
        speaker_title_dict[speakerTitle] = i

    df.speakerTitle.replace(speaker_title_dict, inplace=True)

    state_dict = dict()
    for i, state in enumerate(pd.unique(df['state'])):
        state_dict[state] = i

    df.state.replace(state_dict, inplace=True)

    party_dict = dict()
    for i, party in enumerate(pd.unique(df['party'])):
        party_dict[party] = i

    df.party.replace(party_dict, inplace=True)

    return df

In [0]:
def clean_data_dl(hd_df, col_name):

    hd_df[col_name] = hd_df[col_name].apply(lambda x: " ".join(x.lower() for x in x.split()))

    # Removing tags
    hd_df[col_name] = hd_df[col_name].str.replace('<.*?>','')

    # Removing possible mentions or urls (don't know if it's necessary but might be) 
    hd_df[col_name] = hd_df[col_name].str.replace('@\w+','')
    hd_df[col_name] = hd_df[col_name].str.replace('http.?://[^\s]+[\s]?','')

    # Removing punctuation and symbols
    hd_df[col_name] = hd_df[col_name].str.replace('[^\w\s]', '')
    hd_df[col_name] = hd_df[col_name].apply(lambda x: " ".join(x for x in x.split() if x not in string.punctuation))

    # Removing non alphabetical character
    hd_df[col_name] = hd_df[col_name].apply(lambda x: " ".join(x for x in x.split() if x.isalpha()))

    # Removing characters non longer than 1
    hd_df[col_name] = hd_df[col_name].apply(lambda x: " ".join(x for x in x.split() if len(x) > 1))

    # Removing stopwords
    sw = stopwords.words('english')
    hd_df[col_name] = hd_df[col_name].apply(lambda x: " ".join(x for x in x.split() if x not in sw))

    # Removing digits
    hd_df[col_name] = hd_df[col_name].apply(lambda x: " ".join(x for x in x.split() if not x.isdigit()))

    # Removing words that appear less than 5
    word_freq = pd.Series(' '.join(hd_df[col_name]).split()).value_counts()
    less_freq = word_freq[word_freq < 5]
    hd_df[col_name] = hd_df[col_name].apply(lambda x: " ".join(x for x in x.split() if x not in less_freq))

    # Removing multiple spaces
    hd_df[col_name] = hd_df[col_name].apply(lambda x: x.strip())
    hd_df[col_name] = hd_df[col_name].str.replace(' +',' ')

    # Lemmatization (better than stemmatization imho)
    hd_df[col_name] = hd_df[col_name].apply(lambda x: " ".join([Word(w).lemmatize() for w in x.split()]))


    return hd_df


In [0]:
columns = ['serialNo', 'ID', 'label', 'statement', 'subject', 'speaker', 'speakerTitle', 'state', 'party',
                  'barely-true', 'false', 'half-true', 'mostly-true', 'pants-fire', 'context', 'justification']

train_data = pd.read_csv('train2.tsv', delimiter='\t',names = columns)
test_data = pd.read_csv('test2.tsv', delimiter='\t',names = columns)
val_data = pd.read_csv('val2.tsv', delimiter='\t', names= columns)

train_data = train_data.drop(['ID','serialNo'],axis = 1)
test_data = test_data.drop(['ID','serialNo'],axis = 1)
val_data = val_data.drop(['ID','serialNo'],axis = 1)


In [0]:
train_data.head()

,label,statement,subject,speaker,speakerTitle,state,party,barely-true,false,half-true,mostly-true,pants-fire,context,justification
0,false,Says the Annies List political group supports ...,abortion,dwayne-bohac,State representative,Texas,republican,0.0,1.0,0.0,0.0,0.0,a mailer,That's a premise that he fails to back up. Ann...
1,half-true,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0.0,0.0,1.0,1.0,0.0,a floor speech.,"Surovell said the decline of coal ""started whe..."
2,mostly-true,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,Denver,Obama said he would have voted against the ame...
3,false,Health care reform legislation is likely to ma...,health-care,blog-posting,NaN,NaN,none,7.0,19.0,3.0,5.0,44.0,a news release,The release may have a point that Mikulskis co...
4,half-true,The economic turnaround started at the end of ...,"economy,jobs",charlie-crist,NaN,Florida,democrat,15.0,9.0,20.0,19.0,2.0,an interview on CNN,"Crist said that the economic ""turnaround start..."


In [0]:
train_data.shape

(10240, 14)

In [0]:
train_data = cleaner(train_data,b = False)
test_data = cleaner(test_data,b = False)
val_data = cleaner(val_data,b = False)


In [0]:
train_data.head()

,label,statement,subject,speaker,speakerTitle,state,party,barely-true,false,half-true,mostly-true,pants-fire,context,justification
0,4,says annies list political group supports thir...,abortion,0,0,0,0,0.0,1.0,0.0,0.0,0.0,mailer,that's premise fails back up. annie's list mak...
1,2,when decline coal start? it started natural ga...,"energy,history,job-accomplishments",1,1,1,1,0.0,0.0,1.0,1.0,0.0,floor speech.,"surovell said decline coal ""started natural ga..."
2,1,"hillary clinton agrees john mccain ""by voting ...",foreign-policy,2,2,2,1,70.0,71.0,160.0,163.0,9.0,denver,obama said would voted amendment present. so t...
3,4,health care reform legislation likely mandate ...,health-care,3,3,3,2,7.0,19.0,3.0,5.0,44.0,news releas,the release may point mikulskis comment could ...
4,2,the economic turnaround started end term.,"economy,jobs",4,3,4,1,15.0,9.0,20.0,19.0,2.0,interview cnn,"crist said economic ""turnaround started end te..."


In [0]:
train_data = clean_data_dl(train_data,"statement")
train_data = clean_data_dl(train_data,"subject")
train_data = clean_data_dl(train_data,"context")
train_data = clean_data_dl(train_data,"justification")

test_data = clean_data_dl(test_data,"statement")
test_data = clean_data_dl(test_data,"subject")
test_data = clean_data_dl(test_data,"context")
test_data = clean_data_dl(test_data,"justification")

val_data = clean_data_dl(val_data,"subject")
val_data = clean_data_dl(val_data,"context")
val_data = clean_data_dl(val_data,"justification")

In [0]:
train_data.head()

,label,statement,subject,speaker,speakerTitle,state,party,barely-true,false,half-true,mostly-true,pants-fire,context,justification
0,4,say list political group support abortion demand,abortion,0,0,0,0,0.0,1.0,0.0,0.0,0.0,mailer,thats premise fails back list make comfortable...
1,2,decline coal start started natural gas took st...,,1,1,1,1,0.0,0.0,1.0,1.0,0.0,floor speech,said decline coal started natural gas took sta...
2,1,hillary clinton john mccain voting give george...,foreignpolicy,2,2,2,1,70.0,71.0,160.0,163.0,9.0,denver,obama said would voted amendment present thoug...
3,4,health care reform legislation likely mandate ...,healthcare,3,3,3,2,7.0,19.0,3.0,5.0,44.0,news releas,release may point comment could open door medi...
4,2,economic started end term,economyjobs,4,3,4,1,15.0,9.0,20.0,19.0,2.0,interview cnn,crist said economic turnaround started end ter...


In [0]:
# Machine Learning approches :

In [0]:
## Logistic regression with all features :

In [0]:
def model_logisticregression(train_data, val_data, test_data):
    vectorizer_statement = CountVectorizer(binary=True, stop_words=nltk.corpus.stopwords.words('english'),
                                           lowercase=True, min_df=3, max_df=0.9, max_features=3000)

    vectorizer_subject = CountVectorizer(binary=True, stop_words=nltk.corpus.stopwords.words('english'),
                                         lowercase=True, min_df=3, max_df=0.9, max_features=3000)

    vectorizer_context = CountVectorizer(binary=True, stop_words=nltk.corpus.stopwords.words('english'),
                                         lowercase=True, min_df=3, max_df=0.9, max_features=3000)

    vectorizer_justification = CountVectorizer(binary=True, stop_words=nltk.corpus.stopwords.words('english'),
                                               lowercase=True, min_df=3, max_df=0.9, max_features=3000)

    statement_onehotvector = vectorizer_statement.fit_transform(train_data['statement']).toarray()
    subject_onehotvector = vectorizer_subject.fit_transform(train_data['subject']).toarray()
    context_onehotvector = vectorizer_context.fit_transform(train_data['context']).toarray()
    justification_onehotvector = vectorizer_justification.fit_transform(train_data['justification']).toarray()

    X_train, y_train = pd.concat(
        [pd.DataFrame(statement_onehotvector), pd.DataFrame(subject_onehotvector), train_data.iloc[:, 3:12],
         pd.DataFrame(context_onehotvector), pd.DataFrame(justification_onehotvector)], axis=1), train_data[
                           'label'].astype('int')

    statement_onehotvector = vectorizer_statement.transform(val_data['statement']).toarray()
    subject_onehotvector = vectorizer_subject.transform(val_data['subject']).toarray()
    context_onehotvector = vectorizer_context.transform(val_data['context']).toarray()
    justification_onehotvector = vectorizer_justification.transform(val_data['justification']).toarray()

    X_val, y_val = pd.concat(
        [pd.DataFrame(statement_onehotvector), pd.DataFrame(subject_onehotvector), val_data.iloc[:, 3:12],
         pd.DataFrame(context_onehotvector), pd.DataFrame(justification_onehotvector)], axis=1), val_data[
                       'label'].astype('int')

    statement_onehotvector = vectorizer_statement.transform(test_data['statement']).toarray()
    subject_onehotvector = vectorizer_subject.transform(test_data['subject']).toarray()
    context_onehotvector = vectorizer_context.transform(test_data['context']).toarray()
    justification_onehotvector = vectorizer_justification.transform(test_data['justification']).toarray()

    X_test, y_test = pd.concat(
        [pd.DataFrame(statement_onehotvector), pd.DataFrame(subject_onehotvector), test_data.iloc[:, 3:12],
         pd.DataFrame(context_onehotvector), pd.DataFrame(justification_onehotvector)], axis=1), test_data[
                         'label'].astype('int')

    print(X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape)

    clf = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=250)
    print("\n\n============ Model Summary ============")
    print(clf)

    print("\n\n============ Model Training ============")
    clf = clf.fit(X_train, y_train)

    print("\n\n============ Model Evaluation ============")
    scores = clf.score(X_val, y_val)
    print("Validation data Accuracy:", scores)
    scores = clf.score(X_test, y_test)
    print("Test data Accuracy:", scores)

In [0]:
model_logisticregression(train_data, val_data, test_data)

(10240, 6597) (10240,) (1284, 6597) (1284,) (1267, 6597) (1267,)


============ Model Summary ============
LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=2000,
                   multi_class='multinomial', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)


============ Model Training ============


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)




============ Model Evaluation ============
Validation data Accuracy: 0.3341121495327103
Test data Accuracy: 0.3007103393843725


In [0]:
## Logistic Regression without subject and context :

In [0]:
def model_logisticregression_withoutSC(train_data, val_data, test_data):
    vectorizer_statement = CountVectorizer(binary=True, stop_words=nltk.corpus.stopwords.words('english'),
                                           lowercase=True, min_df=3, max_df=0.9, max_features=3000)

    vectorizer_justification = CountVectorizer(binary=True, stop_words=nltk.corpus.stopwords.words('english'),
                                               lowercase=True, min_df=3, max_df=0.9, max_features=3000)

    statement_onehotvector = vectorizer_statement.fit_transform(train_data['statement']).toarray()
    justification_onehotvector = vectorizer_justification.fit_transform(train_data['justification']).toarray()

    X_train, y_train = pd.concat(
        [pd.DataFrame(statement_onehotvector), train_data.iloc[:, 3:12], pd.DataFrame(justification_onehotvector)], axis=1), train_data[
                           'label'].astype('int')

    statement_onehotvector = vectorizer_statement.transform(val_data['statement']).toarray()
    justification_onehotvector = vectorizer_justification.transform(val_data['justification']).toarray()

    X_val, y_val = pd.concat(
        [pd.DataFrame(statement_onehotvector), val_data.iloc[:, 3:12],pd.DataFrame(justification_onehotvector)], axis=1), val_data[
                       'label'].astype('int')

    statement_onehotvector = vectorizer_statement.transform(test_data['statement']).toarray()
    justification_onehotvector = vectorizer_justification.transform(test_data['justification']).toarray()

    X_test, y_test = pd.concat(
        [pd.DataFrame(statement_onehotvector), test_data.iloc[:, 3:12],pd.DataFrame(justification_onehotvector)], axis=1), test_data[
                         'label'].astype('int')

    print(X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape)

    clf = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=250)
    print("\n\n============ Model Summary ============")
    print(clf)

    print("\n\n============ Model Training ============")
    clf = clf.fit(X_train, y_train)

    print("\n\n============ Model Evaluation ============")
    scores = clf.score(X_val, y_val)
    print("Validation data Accuracy:", scores)
    scores = clf.score(X_test, y_test)
    print("Test data Accuracy:", scores)

In [0]:
model_logisticregression_withoutSC(train_data, val_data, test_data)

(10240, 5790) (10240,) (1284, 5790) (1284,) (1267, 5790) (1267,)


============ Model Summary ============
LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=250,
                   multi_class='multinomial', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)


============ Model Training ============


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)




============ Model Evaluation ============
Validation data Accuracy: 0.34579439252336447
Test data Accuracy: 0.31570639305445936


In [0]:
## Logistic Regression with only SMJ

In [0]:
def model_logisticregression_SMJ(train_data, val_data, test_data):
    vectorizer_statement = CountVectorizer(binary=True, stop_words=nltk.corpus.stopwords.words('english'),
                                           lowercase=True, min_df=3, max_df=0.9, max_features=3000)

    vectorizer_subject = CountVectorizer(binary=True, stop_words=nltk.corpus.stopwords.words('english'),
                                         lowercase=True, min_df=3, max_df=0.9, max_features=3000)

    vectorizer_context = CountVectorizer(binary=True, stop_words=nltk.corpus.stopwords.words('english'),
                                         lowercase=True, min_df=3, max_df=0.9, max_features=3000)

    vectorizer_justification = CountVectorizer(binary=True, stop_words=nltk.corpus.stopwords.words('english'),
                                               lowercase=True, min_df=3, max_df=0.9, max_features=3000)

    statement_onehotvector = vectorizer_statement.fit_transform(train_data['statement']).toarray()
    subject_onehotvector = vectorizer_subject.fit_transform(train_data['subject']).toarray()
    context_onehotvector = vectorizer_context.fit_transform(train_data['context']).toarray()
    justification_onehotvector = vectorizer_justification.fit_transform(train_data['justification']).toarray()

    X_train, y_train = pd.concat(
        [pd.DataFrame(statement_onehotvector), pd.DataFrame(subject_onehotvector),
         pd.DataFrame(context_onehotvector), pd.DataFrame(justification_onehotvector)], axis=1), train_data[
                           'label'].astype('int')

    statement_onehotvector = vectorizer_statement.transform(val_data['statement']).toarray()
    subject_onehotvector = vectorizer_subject.transform(val_data['subject']).toarray()
    context_onehotvector = vectorizer_context.transform(val_data['context']).toarray()
    justification_onehotvector = vectorizer_justification.transform(val_data['justification']).toarray()

    X_val, y_val = pd.concat(
        [pd.DataFrame(statement_onehotvector), pd.DataFrame(subject_onehotvector), 
         pd.DataFrame(context_onehotvector), pd.DataFrame(justification_onehotvector)], axis=1), val_data[
                       'label'].astype('int')

    statement_onehotvector = vectorizer_statement.transform(test_data['statement']).toarray()
    subject_onehotvector = vectorizer_subject.transform(test_data['subject']).toarray()
    context_onehotvector = vectorizer_context.transform(test_data['context']).toarray()
    justification_onehotvector = vectorizer_justification.transform(test_data['justification']).toarray()

    X_test, y_test = pd.concat(
        [pd.DataFrame(statement_onehotvector), pd.DataFrame(subject_onehotvector),
         pd.DataFrame(context_onehotvector), pd.DataFrame(justification_onehotvector)], axis=1), test_data[
                         'label'].astype('int')

    print(X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape)


    clf = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=250)
    print("\n\n============ Model Summary ============")
    print(clf)

    print("\n\n============ Model Training ============")
    clf = clf.fit(X_train, y_train)

    print("\n\n============ Model Evaluation ============")
    scores = clf.score(X_val, y_val)
    print("Validation data Accuracy:", scores)
    scores = clf.score(X_test, y_test)
    print("Test data Accuracy:", scores)

In [0]:
model_logisticregression_SMJ(train_data, val_data, test_data)

(10240, 6588) (10240,) (1284, 6588) (1284,) (1267, 6588) (1267,)


============ Model Summary ============
LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=250,
                   multi_class='multinomial', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)


============ Model Training ============


============ Model Evaluation ============
Validation data Accuracy: 0.2531152647975078
Test data Accuracy: 0.2265193370165746


In [0]:
## Logistic Regression with only SJ

In [0]:
def model_logisticregression_SJ(train_data, val_data, test_data):
    vectorizer_statement = CountVectorizer(binary=True, stop_words=nltk.corpus.stopwords.words('english'),
                                           lowercase=True, min_df=3, max_df=0.9, max_features=3000)

    vectorizer_justification = CountVectorizer(binary=True, stop_words=nltk.corpus.stopwords.words('english'),
                                               lowercase=True, min_df=3, max_df=0.9, max_features=3000)

    statement_onehotvector = vectorizer_statement.fit_transform(train_data['statement']).toarray()
    justification_onehotvector = vectorizer_justification.fit_transform(train_data['justification']).toarray()

    X_train, y_train = pd.concat(
        [pd.DataFrame(statement_onehotvector), pd.DataFrame(justification_onehotvector)], axis=1), train_data[
                           'label'].astype('int')

    statement_onehotvector = vectorizer_statement.transform(val_data['statement']).toarray()
    justification_onehotvector = vectorizer_justification.transform(val_data['justification']).toarray()

    X_val, y_val = pd.concat(
        [pd.DataFrame(statement_onehotvector),pd.DataFrame(justification_onehotvector)], axis=1), val_data[
                       'label'].astype('int')

    statement_onehotvector = vectorizer_statement.transform(test_data['statement']).toarray()
    justification_onehotvector = vectorizer_justification.transform(test_data['justification']).toarray()

    X_test, y_test = pd.concat(
        [pd.DataFrame(statement_onehotvector),pd.DataFrame(justification_onehotvector)], axis=1), test_data[
                         'label'].astype('int')

    print(X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape)

    clf = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=250)
    print("\n\n============ Model Summary ============")
    print(clf)

    print("\n\n============ Model Training ============")
    clf = clf.fit(X_train, y_train)

    print("\n\n============ Model Evaluation ============")
    scores = clf.score(X_val, y_val)
    print("Validation data Accuracy:", scores)
    scores = clf.score(X_test, y_test)
    print("Test data Accuracy:", scores)

In [0]:
model_logisticregression_SJ(train_data, val_data, test_data)

(10240, 5781) (10240,) (1284, 5781) (1284,) (1267, 5781) (1267,)


============ Model Summary ============
LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=250,
                   multi_class='multinomial', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)


============ Model Training ============


============ Model Evaluation ============
Validation data Accuracy: 0.2250778816199377
Test data Accuracy: 0.21625887924230466


In [0]:
# Using ensemling

In [0]:
# Gradient Boosting with all features 

In [0]:
def model_gradientboost(train_data, val_data, test_data):
    vectorizer_statement = CountVectorizer(binary=True, stop_words=nltk.corpus.stopwords.words('english'),
                                           lowercase=True, min_df=3, max_df=0.9, max_features=3000)

    vectorizer_subject = CountVectorizer(binary=True, stop_words=nltk.corpus.stopwords.words('english'),
                                         lowercase=True, min_df=3, max_df=0.9, max_features=3000)

    vectorizer_context = CountVectorizer(binary=True, stop_words=nltk.corpus.stopwords.words('english'),
                                         lowercase=True, min_df=3, max_df=0.9, max_features=3000)

    vectorizer_justification = CountVectorizer(binary=True, stop_words=nltk.corpus.stopwords.words('english'),
                                               lowercase=True, min_df=3, max_df=0.9, max_features=3000)

    statement_onehotvector = vectorizer_statement.fit_transform(train_data['statement']).toarray()
    subject_onehotvector = vectorizer_subject.fit_transform(train_data['subject']).toarray()
    context_onehotvector = vectorizer_context.fit_transform(train_data['context']).toarray()
    justification_onehotvector = vectorizer_justification.fit_transform(train_data['justification']).toarray()

    X_train, y_train = pd.concat(
        [pd.DataFrame(statement_onehotvector), pd.DataFrame(subject_onehotvector), train_data.iloc[:, 3:12],
         pd.DataFrame(context_onehotvector), pd.DataFrame(justification_onehotvector)], axis=1), train_data[
                           'label'].astype('int')

    statement_onehotvector = vectorizer_statement.transform(val_data['statement']).toarray()
    subject_onehotvector = vectorizer_subject.transform(val_data['subject']).toarray()
    context_onehotvector = vectorizer_context.transform(val_data['context']).toarray()
    justification_onehotvector = vectorizer_justification.transform(val_data['justification']).toarray()

    X_val, y_val = pd.concat(
        [pd.DataFrame(statement_onehotvector), pd.DataFrame(subject_onehotvector), val_data.iloc[:, 3:12],
         pd.DataFrame(context_onehotvector), pd.DataFrame(justification_onehotvector)], axis=1), val_data[
                       'label'].astype('int')

    statement_onehotvector = vectorizer_statement.transform(test_data['statement']).toarray()
    subject_onehotvector = vectorizer_subject.transform(test_data['subject']).toarray()
    context_onehotvector = vectorizer_context.transform(test_data['context']).toarray()
    justification_onehotvector = vectorizer_justification.transform(test_data['justification']).toarray()

    X_test, y_test = pd.concat(
        [pd.DataFrame(statement_onehotvector), pd.DataFrame(subject_onehotvector), test_data.iloc[:, 3:12],
         pd.DataFrame(context_onehotvector), pd.DataFrame(justification_onehotvector)], axis=1), test_data[
                         'label'].astype('int')

    print(X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape)

    gb = GradientBoostingClassifier()
    print("\n\n============ Model Summary ============")
    print(gb)

    print("\n\n============ Model Training ============")
    gb = gb.fit(X_train, y_train)

    print("\n\n============ Model Evaluation ============")
    scores = gb.score(X_val, y_val)
    print("Validation data Accuracy:", scores)
    scores = gb.score(X_test, y_test)
    print("Test data Accuracy:", scores)

In [0]:
model_gradientboost(train_data, val_data, test_data)

(10240, 6597) (10240,) (1284, 6597) (1284,) (1267, 6597) (1267,)


============ Model Summary ============
GradientBoostingClassifier(criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='auto',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)


============ Model Training ============


============ Model Evaluation ============
Validation data Accuracy: 0.4501557632398754
Test data Accuracy: 0.4293606945540647


In [0]:
## Gradient boosting without context and subject

In [0]:
def model_gradientboost_withoutSC(train_data, val_data, test_data):
    vectorizer_statement = CountVectorizer(binary=True, stop_words=nltk.corpus.stopwords.words('english'),
                                           lowercase=True, min_df=3, max_df=0.9, max_features=3000)

    vectorizer_justification = CountVectorizer(binary=True, stop_words=nltk.corpus.stopwords.words('english'),
                                               lowercase=True, min_df=3, max_df=0.9, max_features=3000)

    statement_onehotvector = vectorizer_statement.fit_transform(train_data['statement']).toarray()
    justification_onehotvector = vectorizer_justification.fit_transform(train_data['justification']).toarray()

    X_train, y_train = pd.concat(
        [pd.DataFrame(statement_onehotvector), train_data.iloc[:, 3:12], pd.DataFrame(justification_onehotvector)], axis=1), train_data[
                           'label'].astype('int')

    statement_onehotvector = vectorizer_statement.transform(val_data['statement']).toarray()
    justification_onehotvector = vectorizer_justification.transform(val_data['justification']).toarray()

    X_val, y_val = pd.concat(
        [pd.DataFrame(statement_onehotvector), val_data.iloc[:, 3:12],pd.DataFrame(justification_onehotvector)], axis=1), val_data[
                       'label'].astype('int')

    statement_onehotvector = vectorizer_statement.transform(test_data['statement']).toarray()
    justification_onehotvector = vectorizer_justification.transform(test_data['justification']).toarray()

    X_test, y_test = pd.concat(
        [pd.DataFrame(statement_onehotvector), test_data.iloc[:, 3:12],pd.DataFrame(justification_onehotvector)], axis=1), test_data[
                         'label'].astype('int')

    print(X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape)

    gb = GradientBoostingClassifier()
    print("\n\n============ Model Summary ============")
    print(gb)

    print("\n\n============ Model Training ============")
    gb = gb.fit(X_train, y_train)

    print("\n\n============ Model Evaluation ============")
    scores = gb.score(X_val, y_val)
    print("Validation data Accuracy:", scores)
    scores = gb.score(X_test, y_test)
    print("Test data Accuracy:", scores)

In [0]:
model_gradientboost_withoutSC(train_data, val_data, test_data)

(10240, 5790) (10240,) (1284, 5790) (1284,) (1267, 5790) (1267,)


============ Model Summary ============
GradientBoostingClassifier(criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='auto',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)


============ Model Training ============


============ Model Evaluation ============
Validation data Accuracy: 0.4649532710280374
Test data Accuracy: 0.4238358326756117


In [0]:
## Gradient Boosting with only SMJ

In [0]:
def model_gradientboost_SMJ(train_data, val_data, test_data):
    vectorizer_statement = CountVectorizer(binary=True, stop_words=nltk.corpus.stopwords.words('english'),
                                           lowercase=True, min_df=3, max_df=0.9, max_features=3000)

    vectorizer_subject = CountVectorizer(binary=True, stop_words=nltk.corpus.stopwords.words('english'),
                                         lowercase=True, min_df=3, max_df=0.9, max_features=3000)

    vectorizer_context = CountVectorizer(binary=True, stop_words=nltk.corpus.stopwords.words('english'),
                                         lowercase=True, min_df=3, max_df=0.9, max_features=3000)

    vectorizer_justification = CountVectorizer(binary=True, stop_words=nltk.corpus.stopwords.words('english'),
                                               lowercase=True, min_df=3, max_df=0.9, max_features=3000)

    statement_onehotvector = vectorizer_statement.fit_transform(train_data['statement']).toarray()
    subject_onehotvector = vectorizer_subject.fit_transform(train_data['subject']).toarray()
    context_onehotvector = vectorizer_context.fit_transform(train_data['context']).toarray()
    justification_onehotvector = vectorizer_justification.fit_transform(train_data['justification']).toarray()

    X_train, y_train = pd.concat(
        [pd.DataFrame(statement_onehotvector), pd.DataFrame(subject_onehotvector),
         pd.DataFrame(context_onehotvector), pd.DataFrame(justification_onehotvector)], axis=1), train_data[
                           'label'].astype('int')

    statement_onehotvector = vectorizer_statement.transform(val_data['statement']).toarray()
    subject_onehotvector = vectorizer_subject.transform(val_data['subject']).toarray()
    context_onehotvector = vectorizer_context.transform(val_data['context']).toarray()
    justification_onehotvector = vectorizer_justification.transform(val_data['justification']).toarray()

    X_val, y_val = pd.concat(
        [pd.DataFrame(statement_onehotvector), pd.DataFrame(subject_onehotvector), 
         pd.DataFrame(context_onehotvector), pd.DataFrame(justification_onehotvector)], axis=1), val_data[
                       'label'].astype('int')

    statement_onehotvector = vectorizer_statement.transform(test_data['statement']).toarray()
    subject_onehotvector = vectorizer_subject.transform(test_data['subject']).toarray()
    context_onehotvector = vectorizer_context.transform(test_data['context']).toarray()
    justification_onehotvector = vectorizer_justification.transform(test_data['justification']).toarray()

    X_test, y_test = pd.concat(
        [pd.DataFrame(statement_onehotvector), pd.DataFrame(subject_onehotvector),
         pd.DataFrame(context_onehotvector), pd.DataFrame(justification_onehotvector)], axis=1), test_data[
                         'label'].astype('int')

    print(X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape)


    gb = GradientBoostingClassifier()
    print("\n\n============ Model Summary ============")
    print(gb)

    print("\n\n============ Model Training ============")
    gb = gb.fit(X_train, y_train)

    print("\n\n============ Model Evaluation ============")
    scores = gb.score(X_val, y_val)
    print("Validation data Accuracy:", scores)
    scores = gb.score(X_test, y_test)
    print("Test data Accuracy:", scores)

In [0]:
model_gradientboost_SMJ(train_data, val_data, test_data)%%!

(10240, 6588) (10240,) (1284, 6588) (1284,) (1267, 6588) (1267,)


============ Model Summary ============
GradientBoostingClassifier(criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='auto',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)


============ Model Training ============


============ Model Evaluation ============
Validation data Accuracy: 0.2484423676012461
Test data Accuracy: 0.2494080505130229


In [0]:
## Gradient Boosting with only SJ

In [0]:
def model_gradientboost_SJ(train_data, val_data, test_data):
    vectorizer_statement = CountVectorizer(binary=True, stop_words=nltk.corpus.stopwords.words('english'),
                                           lowercase=True, min_df=3, max_df=0.9, max_features=3000)

    vectorizer_justification = CountVectorizer(binary=True, stop_words=nltk.corpus.stopwords.words('english'),
                                               lowercase=True, min_df=3, max_df=0.9, max_features=3000)

    statement_onehotvector = vectorizer_statement.fit_transform(train_data['statement']).toarray()
    justification_onehotvector = vectorizer_justification.fit_transform(train_data['justification']).toarray()

    X_train, y_train = pd.concat(
        [pd.DataFrame(statement_onehotvector), train_data.iloc[:, 3:12], pd.DataFrame(justification_onehotvector)], axis=1), train_data[
                           'label'].astype('int')

    statement_onehotvector = vectorizer_statement.transform(val_data['statement']).toarray()
    justification_onehotvector = vectorizer_justification.transform(val_data['justification']).toarray()

    X_val, y_val = pd.concat(
        [pd.DataFrame(statement_onehotvector), val_data.iloc[:, 3:12],pd.DataFrame(justification_onehotvector)], axis=1), val_data[
                       'label'].astype('int')

    statement_onehotvector = vectorizer_statement.transform(test_data['statement']).toarray()
    justification_onehotvector = vectorizer_justification.transform(test_data['justification']).toarray()

    X_test, y_test = pd.concat(
        [pd.DataFrame(statement_onehotvector), test_data.iloc[:, 3:12],pd.DataFrame(justification_onehotvector)], axis=1), test_data[
                         'label'].astype('int')

    print(X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape)

    gb = GradientBoostingClassifier()
    print("\n\n============ Model Summary ============")
    print(gb)

    print("\n\n============ Model Training ============")
    gb = gb.fit(X_train, y_train)

    print("\n\n============ Model Evaluation ============")
    scores = gb.score(X_val, y_val)
    print("Validation data Accuracy:", scores)
    scores = gb.score(X_test, y_test)
    print("Test data Accuracy:", scores)

In [0]:
model_gradientboost_SJ(train_data, val_data, test_data)

(10240, 5790) (10240,) (1284, 5790) (1284,) (1267, 5790) (1267,)


============ Model Summary ============
GradientBoostingClassifier(criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='auto',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)


============ Model Training ============


============ Model Evaluation ============
Validation data Accuracy: 0.4602803738317757
Test data Accuracy: 0.4246250986582478


In [0]:
# Deep Learning Approach 

In [0]:
# Binary Classification

In [0]:
columns = ['serialNo', 'ID', 'label', 'statement', 'subject', 'speaker', 'speakerTitle', 'state', 'party',
                  'barely-true', 'false', 'half-true', 'mostly-true', 'pants-fire', 'context', 'justification']

train_data = pd.read_csv('train2.tsv', delimiter='\t',names = columns)
test_data = pd.read_csv('test2.tsv', delimiter='\t',names = columns)
val_data = pd.read_csv('val2.tsv', delimiter='\t', names= columns)

train_data = train_data.drop(['ID','serialNo'],axis = 1)
test_data = test_data.drop(['ID','serialNo'],axis = 1)
val_data = val_data.drop(['ID','serialNo'],axis = 1)

train_data = cleaner(train_data,b = True)
test_data = cleaner(test_data,b = True)
val_data = cleaner(val_data,b = True)

train_data = clean_data_dl(train_data,"statement")
train_data = clean_data_dl(train_data,"subject")
train_data = clean_data_dl(train_data,"context")
train_data = clean_data_dl(train_data,"justification")

test_data = clean_data_dl(test_data,"statement")
test_data = clean_data_dl(test_data,"subject")
test_data = clean_data_dl(test_data,"context")
test_data = clean_data_dl(test_data,"justification")

val_data = clean_data_dl(val_data,"subject")
val_data = clean_data_dl(val_data,"context")
val_data = clean_data_dl(val_data,"justification")

In [0]:
## Logistic regression with all features :

In [0]:
model_logisticregression(train_data, val_data, test_data)

(10240, 6589) (10240,) (1284, 6589) (1284,) (1267, 6589) (1267,)


============ Model Summary ============
LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=2000,
                   multi_class='multinomial', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)


============ Model Training ============


============ Model Evaluation ============
Validation data Accuracy: 0.5771028037383178
Test data Accuracy: 0.5674822415153907


In [0]:
## Logistic Regression without subject and context :

In [0]:
model_logisticregression_withoutSC(train_data, val_data, test_data)

(10240, 5791) (10240,) (1284, 5791) (1284,) (1267, 5791) (1267,)


============ Model Summary ============
GradientBoostingClassifier(criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='auto',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)


============ Model Training ============


============ Model Evaluation ============
Validation data Accuracy: 0.7118380062305296
Test data Accuracy: 0.7411207576953434


In [0]:
## Logistic Regression with only SMJ :

In [0]:
model_logisticregression_SMJ(train_data, val_data, test_data)

(10240, 6589) (10240,) (1284, 6589) (1284,) (1267, 6589) (1267,)


============ Model Summary ============
LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=250,
                   multi_class='multinomial', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)


============ Model Training ============


============ Model Evaluation ============
Validation data Accuracy: 0.5771028037383178
Test data Accuracy: 0.5674822415153907


In [0]:
## Logistic Regression with only SJ:

In [0]:
model_logisticregression_SJ(train_data, val_data, test_data)

(10240, 5782) (10240,) (1284, 5782) (1284,) (1267, 5782) (1267,)


============ Model Summary ============
LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=250,
                   multi_class='multinomial', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)


============ Model Training ============


============ Model Evaluation ============
Validation data Accuracy: 0.572429906542056
Test data Accuracy: 0.569060773480663


In [0]:
## Gradient Boosting with all features :

In [0]:
model_gradientboost(train_data, val_data, test_data)

(10240, 6598) (10240,) (1284, 6598) (1284,) (1267, 6598) (1267,)


============ Model Summary ============
GradientBoostingClassifier(criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='auto',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)


============ Model Training ============


============ Model Evaluation ============
Validation data Accuracy: 0.7048286604361371
Test data Accuracy: 0.744277821625888


In [0]:
## Gradient Boosting without subject and context:

In [0]:
model_gradientboost_withoutSC(train_data, val_data, test_data)

(10240, 5791) (10240,) (1284, 5791) (1284,) (1267, 5791) (1267,)


============ Model Summary ============
GradientBoostingClassifier(criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='auto',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)


============ Model Training ============


============ Model Evaluation ============
Validation data Accuracy: 0.7118380062305296
Test data Accuracy: 0.7411207576953434


In [0]:
## Gradient Boosting with only SMJ :

In [0]:
model_gradientboost_SMJ(train_data, val_data, test_data)

(10240, 6589) (10240,) (1284, 6589) (1284,) (1267, 6589) (1267,)


============ Model Summary ============
GradientBoostingClassifier(criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='auto',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)


============ Model Training ============


============ Model Evaluation ============
Validation data Accuracy: 0.5654205607476636
Test data Accuracy: 0.590370955011839


In [0]:
## Gradient Boosting with only SJ :

In [0]:
model_gradientboost_SJ(train_data, val_data, test_data)

(10240, 5791) (10240,) (1284, 5791) (1284,) (1267, 5791) (1267,)


============ Model Summary ============
GradientBoostingClassifier(criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='auto',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)


============ Model Training ============


============ Model Evaluation ============
Validation data Accuracy: 0.7118380062305296
Test data Accuracy: 0.7411207576953434
